

# Train YOLO Object Detection on a Custom Dataset

---


Made to train on Google colab, with the dataset stored in google drive, in zip form

*>>* If you want to run locally, you will need to change the paths to the dataset (in PARAMS) and the output folder (last cell in this notebook) and remove all the google.colab imports and uses

## PARAMS

In [1]:
model_base = "yolo11n.pt"
export_formats = ["onnx", "openvino", "torchscript"]
export_zip_name = "trained_model.zip"
training_params = {
    "epochs" : 1000,
    "batch" : 0.55,
    "cache" : True,
    "imgsz" : 640,
    "classes" : None,
}

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
dataset_zip = "/content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip"

**ALWAYS GO CHECK THE PATH IN THE BEFORE LAST CELL, BY DEFAULT THIS WILL SAVE THE MODELS IN RUNS>DETECT>TRAIN2 !**

## Tutorial
The basic idea is :


```
from ultralytics import YOLO

model = YOLO("yolov10s.pt") # We are going off the pretrained model, not starting from scratch

model.train(data="dataset_folder/data.yaml", epochs=1000, batch=-1, device=0, cache=True, classes=None, patience=50)
# A YOLO dataset is a folder with a data.yaml file giving the params to the model
# epochs = 1000 should ensure the model keeps on training until it judges itself to have attained its best possible performance
# batch = -1 allows the model engine to autodetermine the best batching size (parallelizing inference). Use 0.55 (55%) if VRAM runs out (lower if still runs out)
# device=0 ensures that the model runs on the first GPU available (if more than one : device=[0,1,...]
# device=cpu would allow training on cpu if no GPU available
# by default this script runs on first available gpu, defaults to cpu if none
#cache=True means the images are cached instead of loaded from disk on each pass (make sure enough RAM available)'
# Classes = None, means training on all classes, classes=list[int] will only train on the classes in the list

```

After this process, the trained model is at `runs/detect/train/weights/best.pt`


## Before  start

make sure we have access to GPU

In [4]:
from tensorflow.test import gpu_device_name

DEVICE = gpu_device_name()
if len(DEVICE) > 0:
  DEVICE = DEVICE[12:]
  ! nvidia-smi
else:
  DEVICE = "cpu"

Mon Feb 10 02:42:04 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   54C    P0             17W /   72W |     189MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import os
HOME = os.getcwd()
print(HOME)

/content


## Install YOLO


In [6]:
!pip install ultralytics

In [7]:
from IPython.display import display, Image, clear_output
clear_output()
!yolo check

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Setup complete ✅ (12 CPUs, 53.0 GB RAM, 33.2/112.6 GB disk)

OS                  Linux-6.1.85+-x86_64-with-glibc2.35
Environment         Colab
Python              3.11.11
Install             pip
RAM                 52.96 GB
Disk                33.2/112.6 GB
CPU                 Intel Xeon 2.20GHz
CPU count           12
GPU                 NVIDIA L4, 22693MiB
GPU count           1
CUDA                12.4

numpy               ✅ 1.26.4<=2.1.1,>=1.23.0
matplotlib          ✅ 3.10.0>=3.3.0
opencv-python       ✅ 4.11.0.86>=4.6.0
pillow              ✅ 11.1.0>=7.1.2
pyyaml              ✅ 6.0.2>=5.3.1
requests            ✅ 2.32.3>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.5.1+cu124>=1.8.0
torch               ✅ 2.5.1+cu124!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision         ✅ 0.20.1+cu124>=0.9.0
tqdm                ✅ 4.67.1>=4.64.0
psutil              ✅ 5.9.5
py-cpuinfo          ✅ 9

In [8]:
#!pip install roboflow

#from roboflow import Roboflow
#rf = Roboflow(api_key="obcyD7henw5LjBBuINZP")
#project = rf.workspace("projet-industrie").project("traffic-counting-singapore")
#version = project.version(3)
#dataset = version.download("yolov11")

In [9]:
from ultralytics import YOLO
import wandb
wandb.init(mode="disabled")

# Load dataset

In [10]:
from ultralytics.utils.downloads import unzip_file

dataset_dir = unzip_file(dataset_zip, path="/content/datasets")
print(f'{dataset_dir=}')
files = os.listdir(dataset_dir)
print(f'{files=}')
dataset_yaml = next((f for f in files if f.endswith('.yaml')), None)
dataset_yaml = os.path.join(dataset_dir, dataset_yaml)
print(f'{dataset_yaml=}')

WARNING ⚠️ Skipping /content/drive/MyDrive/Colab Notebooks/datasets/dataset.zip unzip as destination directory /content/datasets/dataset is not empty.
dataset_dir=PosixPath('/content/datasets/dataset')
files=['val', 'test', 'data.yaml', 'train']
dataset_yaml='/content/datasets/dataset/data.yaml'


## Custom Training

In [ ]:
model = YOLO(model_base)

results = model.train(data=dataset_yaml, epochs=training_params['epochs'], batch=training_params['batch'], device=DEVICE, cache=training_params['cache'], imgsz=training_params['imgsz'], classes=training_params['classes'])
for format in export_formats:
  model.export(format=format)

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/datasets/dataset/data.yaml, epochs=1000, time=None, patience=100, batch=0.55, imgsz=640, save=True, save_period=-1, cache=True, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True,

train: Scanning /content/datasets/dataset/train/labels... 533 images, 0 backgrounds, 0 corrupt: 100%|██████████| 533/533 [00:00<00:00, 1194.36it/s]

train: New cache created: /content/datasets/dataset/train/labels.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (0.3GB RAM): 100%|██████████| 533/533 [00:00<00:00, 701.16it/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   381/1000      22.1G     0.8167     0.4805     0.8296       3647        640: 100%|██████████| 6/6 [00:04<00:00,  1.28it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.42it/s]

                   all        133       5342      0.785      0.601      0.655      0.474



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


   382/1000      19.2G      0.792      0.473     0.8278       6992        640:  67%|██████▋   | 4/6 [00:04<00:01,  1.21it/s]

In [ ]:
# Validate the model
metrics = model.val(save_json=True)  # no arguments needed, dataset and settings remembered
metrics.box.map  # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps  # a list contains map50-95 of each category

#this will create a new folder with the results of this val (usually train12, train22, train32 etc..), it can be discarded as they are also included in the original training folder

# Download results

In [ ]:
from ultralytics.utils.downloads import zip_directory
zipped_res = zip_directory('/content/runs/detect/train2')

In [ ]:
from shutil import copy2
copy2(zipped_res, os.path.join('/content/drive/MyDrive/Colab Notebooks/', export_zip_name))